# Install Packages

In [11]:
import random
import sys
from time import time
from collections import deque, defaultdict, namedtuple
import numpy as np
import pandas as pd
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
%matplotlib inline

#plt.style.use('seaborn')
plt.style.use('fivethirtyeight')

# Create Gym Environment

In [12]:
env = gym.make('LunarLander-v2')
state = env.reset(seed=42)
print(env.action_space)
print(env.observation_space)

Discrete(4)
Box([-1.5       -1.5       -5.        -5.        -3.1415927 -5.
 -0.        -0.       ], [1.5       1.5       5.        5.        3.1415927 5.        1.
 1.       ], (8,), float32)


# Random Action Agent

In [13]:
max_episodes = 100

In [14]:
scores = []
actions = range(env.action_space.n)
for i in range(1, max_episodes+1):
    state = env.reset()
    score = 0
    while True:
        action = np.random.choice(actions)
        state, reward, done, info, a = env.step(action)
        score += reward
        if done:
            if i % 20 == 0:
                print('Episode {},  score: {}'.format(i, score))
            break
    
    scores.append(score)

Episode 20,  score: -258.70856078524105
Episode 40,  score: -350.61553513889
Episode 60,  score: -121.87785834165237
Episode 80,  score: -321.29790443792217
Episode 100,  score: -608.9212950532196


In [15]:
print(scores)

[-122.19892403151773, -368.3027877536328, -122.2036863608206, -93.51048679431742, -107.31071501143722, -227.90984406783937, -327.7786412968537, -89.38495217938943, -364.9156544409085, -95.40532264793222, -169.16145918427873, -145.97884051560186, -380.3894154200558, -206.70551911114853, -62.68741992754673, -298.73152702056416, -88.71997157626627, -152.27241418508567, -94.21331485863766, -258.70856078524105, -81.28704865234117, -183.56976364382916, -109.7067229910871, -107.24179884683889, -261.3675154277838, -309.2858481426118, -73.83017009804234, -102.66692103977672, -119.77861615072854, -134.43829529289954, -330.2966102843151, -143.94685575641392, -214.48815992767663, -95.59371707419274, -135.27662019472962, -246.61329978624303, -58.80755636638108, -289.58423588311007, -149.13060803044849, -350.61553513889, -208.39426256839823, -101.34678683723939, -157.64608123484328, -127.59085216165417, -152.2302628769002, -88.8255330952528, -305.59667682224244, -180.22452145976246, -425.26663207254

# Average score

In [16]:
# Average score
print('Average score of random agent over {} episodes: {:.2f}'.format(max_episodes, np.mean(scores)))

Average score of random agent over 100 episodes: -183.41
